# Import

In [394]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy.fft import rfft, irfft, rfftfreq

In [395]:
import torch

In [396]:
import torch.nn as nn

In [397]:
import torch.nn.functional as F

In [398]:
from sklearn.model_selection import train_test_split

In [399]:
from torch.utils.data import  TensorDataset, DataLoader

In [400]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# Data Load

In [401]:
path_fault2_ex = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal2/external'
path_fault2_in = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal2/internal'

path_fault1_ex = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal1/external'
path_fault1_in = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal1/internal'

path_normal_ex = 'C:/Users/baek/Desktop/NURO/data_cycle/normal/external'
path_normal_in = 'C:/Users/baek/Desktop/NURO/data_cycle/normal/internal'

In [402]:
dat_f1 = [[] for i in range(0,2)]
dat_f2 = [[] for i in range(0,2)]
dat_n = []

In [403]:
for i in range(len(os.listdir(path_fault1_ex))):
    temp =pd.read_csv(path_fault1_ex + '/'+ os.listdir(path_fault1_ex)[i], sep = '\t' )
    dat_f1[0].append(temp)
    
    
for i in range(len(os.listdir(path_fault1_in))):
    temp = pd.read_csv(path_fault1_in + '/'+os.listdir(path_fault1_in)[i], sep = ',' , low_memory=False)
    dat_f1[1].append(temp)

    
for i in range(len(os.listdir(path_fault2_ex))):
    temp =pd.read_csv(path_fault2_ex + '/'+ os.listdir(path_fault2_ex)[i], sep = '\t' )
    dat_f2[0].append(temp)
    
    
for i in range(len(os.listdir(path_fault2_in))):
    temp = pd.read_csv(path_fault2_in + '/'+os.listdir(path_fault2_in)[i], sep = ',' , low_memory=False)
    dat_f2[1].append(temp)
    
    
for i in range(len(os.listdir(path_normal_ex))):
    temp =pd.read_csv(path_normal_ex + '/'+os.listdir(path_normal_ex)[i], sep = '\t' )
    dat_n.append(temp)

In [475]:
data_f1 = []
data_f2 = []
data_n = []

for i in range(len(dat_f1[1])):
    data_n.append(dat_f1[1][i].loc[dat_f1[1][i]['CoreSN'] == 'core0'+str(1),:].reset_index().drop(labels = 'index', axis=1))


for i in range(len(dat_f1[1])):
    data_f1.append(dat_f1[1][i].loc[dat_f1[1][i]['CoreSN'] == 'core0'+str(2),:].reset_index().drop(labels = 'index', axis=1))

    
for i in range(len(dat_f2[1])):
    data_f2.append(dat_f2[1][i].loc[dat_f2[1][i]['CoreSN'] == 'core0'+str(2),:].reset_index().drop(labels = 'index', axis=1))


In [476]:
d_n = []
d_f1 = []
d_f2 = []


for i in range(0,5):
    for j in range(len(data_n[i])//511):
        d_n.append(data_n[i].loc[511*j:511*(j+1),'ActualTorque'])

for i in range(0,5):
    for j in range(len(data_f1[i])//511):
        d_f1.append(data_f1[i].loc[511*j:511*(j+1),'ActualTorque'])
        
for i in range(0,5):
    for j in range(len(data_f2[i])//511):
        d_f2.append(data_f2[i].loc[511*j:511*(j+1),'ActualTorque'])


In [477]:
label_n = [0 for i in range(len(d_n))]
label_f1 = [1 for i in range(len(d_f1))]
label_f2 = [2 for i in range(len(d_f2))]


d_n.extend(d_f1)
label_n.extend(label_f1)

d_n.extend(d_f2)
label_n.extend(label_f2)

In [666]:
X_train, X_valid, y_train, y_valid = train_test_split(d_n,label_n,test_size=0.2, random_state=42)

for i in range(len(X_train)):
    X_train[i] = np.array(X_train[i])
    
for i in range(len(X_valid)):
    X_valid[i] = np.array(X_valid[i])
    
X_valid = np.array(X_valid)
X_train = np.array(X_train)
y_valid = np.array(y_valid)
y_train = np.array(y_train)

X_valid = X_valid.reshape(len(X_valid),1,512)
X_train = X_train.reshape(len(X_train),1,512)


In [667]:
X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)

X_train = X_train.view([-1,1, 512])
X_valid = X_valid.view([-1,1, 512])

y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

In [668]:
Train = TensorDataset(X_train,y_train)
Valid = TensorDataset(X_valid, y_valid)



Train_loader = DataLoader(Train, batch_size=32, shuffle=True)
Valid_loader = DataLoader(Valid, batch_size=32, shuffle=True)

# Model

In [669]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(1,32,kernel_size = 2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool1d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(32,64, kernel_size =2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool1d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(64,128, kernel_size =2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool1d(2)
        )
    
        self.fc1 = nn.Linear(8064,1000, bias = True)
        self.fc2 = nn.Linear(1000,100, bias = True)
        self.fc3 = nn.Linear(100,3, bias = True)
        torch.nn.init.xavier_uniform_(self.fc3.weight)
#Convolution - Batch Normalization - Activation - Dropout - Pooling
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        #print(out.shape)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [670]:
# instantiate CNN model
model = CNN().to(device)

# Train & Valid

In [671]:
# parameters
learning_rate = 0.001
training_epochs = 200
batch_size = 32

In [672]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [673]:
# train my model
total_train_batch = len(Train_loader)
total_valid_batch = len(Valid_loader)

print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    avg_acc = 0
    for X, Y in Train_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_train_batch
        pred = hypothesis.argmax(dim = 1)
        avg_acc += torch.eq(pred, Y).sum().float().item()
    avg_acc =avg_acc/ 898
    with torch.no_grad():
        avg_val_loss = 0
        val_loss = 0
        avg_val_acc = 0
        for X_val, Y_val in Valid_loader:
            X_val = X_val.to(device)
            Y_val = Y_val.to(device)
            
            y_val_pred = model(X_val)
            val_step_loss = criterion(y_val_pred,Y_val)
            avg_val_loss += val_step_loss / total_valid_batch
            pred_val = y_val_pred.argmax(dim = 1)
            avg_val_acc += torch.eq(pred_val, Y_val).sum().float().item()
        avg_val_acc = avg_val_acc/ 225
    print('[Epoch: {:>4}] train_cost = {:>.9}, valid_cost = {:>.9}'.format(epoch + 1, avg_cost,avg_val_loss))
    print('[Epoch: {:>4}] train_acc = {:>.9}, valid_acc = {:>.9}'.format(epoch + 1, avg_acc, avg_val_acc))


print('Learning Finished!')


Learning started. It takes sometime.
[Epoch:    1] train_cost = 9.37433624, valid_cost = 6.70257425
[Epoch:    1] train_acc = 0.420935412, valid_acc = 0.342222222
[Epoch:    2] train_cost = 2.86983371, valid_cost = 1.26878119
[Epoch:    2] train_acc = 0.477728285, valid_acc = 0.617777778
[Epoch:    3] train_cost = 0.898505569, valid_cost = 1.55584145
[Epoch:    3] train_acc = 0.639198218, valid_acc = 0.533333333
[Epoch:    4] train_cost = 0.862832725, valid_cost = 0.782449007
[Epoch:    4] train_acc = 0.646993318, valid_acc = 0.715555556
[Epoch:    5] train_cost = 0.808022976, valid_cost = 0.709853292
[Epoch:    5] train_acc = 0.667037862, valid_acc = 0.742222222
[Epoch:    6] train_cost = 0.616550684, valid_cost = 0.748764157
[Epoch:    6] train_acc = 0.778396437, valid_acc = 0.684444444
[Epoch:    7] train_cost = 0.619735956, valid_cost = 1.24737442
[Epoch:    7] train_acc = 0.802895323, valid_acc = 0.644444444
[Epoch:    8] train_cost = 0.870741248, valid_cost = 1.35408199
[Epoch:  

[Epoch:   67] train_cost = 0.543908596, valid_cost = 4.45637131
[Epoch:   67] train_acc = 0.893095768, valid_acc = 0.493333333
[Epoch:   68] train_cost = 0.730573893, valid_cost = 1.5090853
[Epoch:   68] train_acc = 0.831848552, valid_acc = 0.835555556
[Epoch:   69] train_cost = 0.488435686, valid_cost = 1.95518291
[Epoch:   69] train_acc = 0.95545657, valid_acc = 0.626666667
[Epoch:   70] train_cost = 1.03952813, valid_cost = 7.31361771
[Epoch:   70] train_acc = 0.861915367, valid_acc = 0.328888889
[Epoch:   71] train_cost = 0.971214116, valid_cost = 3.00490046
[Epoch:   71] train_acc = 0.834075724, valid_acc = 0.76
[Epoch:   72] train_cost = 0.494855553, valid_cost = 0.664452732
[Epoch:   72] train_acc = 0.885300668, valid_acc = 0.808888889
[Epoch:   73] train_cost = 0.185414836, valid_cost = 0.855958223
[Epoch:   73] train_acc = 0.945434298, valid_acc = 0.755555556
[Epoch:   74] train_cost = 0.324841827, valid_cost = 0.473781586
[Epoch:   74] train_acc = 0.897550111, valid_acc = 0.8

[Epoch:  132] train_cost = 0.233972788, valid_cost = 0.416713446
[Epoch:  132] train_acc = 0.976614699, valid_acc = 0.911111111
[Epoch:  133] train_cost = 0.197876528, valid_cost = 0.775957227
[Epoch:  133] train_acc = 0.923162584, valid_acc = 0.862222222
[Epoch:  134] train_cost = 0.099829033, valid_cost = 0.655138314
[Epoch:  134] train_acc = 0.958797327, valid_acc = 0.911111111
[Epoch:  135] train_cost = 0.0538870208, valid_cost = 0.577011645
[Epoch:  135] train_acc = 0.977728285, valid_acc = 0.857777778
[Epoch:  136] train_cost = 0.0488972105, valid_cost = 0.525780737
[Epoch:  136] train_acc = 0.982182628, valid_acc = 0.893333333
[Epoch:  137] train_cost = 0.101380751, valid_cost = 0.552458584
[Epoch:  137] train_acc = 0.981069042, valid_acc = 0.871111111
[Epoch:  138] train_cost = 0.283459902, valid_cost = 0.668409824
[Epoch:  138] train_acc = 0.906458797, valid_acc = 0.822222222
[Epoch:  139] train_cost = 0.161252081, valid_cost = 0.636727452
[Epoch:  139] train_acc = 0.952115813

[Epoch:  198] train_cost = 0.184293389, valid_cost = 2.22641993
[Epoch:  198] train_acc = 0.977728285, valid_acc = 0.888888889
[Epoch:  199] train_cost = 0.183608949, valid_cost = 1.25420451
[Epoch:  199] train_acc = 0.949888641, valid_acc = 0.888888889
[Epoch:  200] train_cost = 0.0669900924, valid_cost = 0.507252634
[Epoch:  200] train_acc = 0.975501114, valid_acc = 0.911111111
Learning Finished!


# Test 

In [721]:
d_n_test = []
d_f1_test = []
d_f2_test = []


for i in range(5,9):
    for j in range(len(data_n[i])//511):
        d_n_test.append(data_n[i].loc[511*j:511*(j+1),'ActualTorque'])

for i in range(5,9):
    for j in range(len(data_f1[i])//511):
        d_f1_test.append(data_f1[i].loc[511*j:511*(j+1),'ActualTorque'])
        
for i in range(5,9):
    for j in range(len(data_f2[i])//511):
        d_f2_test.append(data_f2[i].loc[511*j:511*(j+1),'ActualTorque'])


In [722]:
label_n_test = [0 for i in range(len(d_n_test))]
label_f1_test = [1 for i in range(len(d_f1_test))]
label_f2_test = [2 for i in range(len(d_f2_test))]


d_n_test.extend(d_f1_test)
label_n_test.extend(label_f1_test)

d_n_test.extend(d_f2_test)
label_n_test.extend(label_f2_test)

In [723]:
X_test, X_bu, y_test, y_bu = train_test_split(d_n_test,label_n_test,test_size=0.1, random_state=42)

for i in range(len(X_test)):
    X_test[i] = np.array(X_test[i])
    
for i in range(len(X_bu)):
    X_bu[i] = np.array(X_bu[i])
    
X_test = np.array(X_test)
X_bu = np.array(X_bu)
y_bu = np.array(y_bu)
y_test = np.array(y_test)

X_bu = X_bu.reshape(len(X_bu),1,512)
X_test = X_test.reshape(len(X_test),1,512)


In [724]:
X_test = torch.FloatTensor(X_test)
X_test = X_test.view([-1,1, 512])

y_test = torch.LongTensor(y_test)

In [725]:
Test = TensorDataset(X_test, y_test)
Test_loader = DataLoader(Test, batch_size=32, shuffle=True)

In [726]:
total_test_batch = len(Test_loader)

with torch.no_grad():
    avg_test_loss = 0
    test_loss = 0
    avg_test_acc = 0
    for X_test, Y_test in Test_loader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)

        y_test_pred = model(X_test)
        test_step_loss = criterion(y_test_pred,Y_test)
        avg_test_loss += test_step_loss / total_test_batch
        pred_test = y_test_pred.argmax(dim = 1)
        avg_test_acc += torch.eq(pred_test, Y_test).sum().float().item()
    avg_test_acc = avg_test_acc/ 913
print('TEST_cost = {:>.9}'.format(avg_test_loss))
print('TEST_acc = {:>.9}'.format( avg_test_acc))

TEST_cost = 0.86042881
TEST_acc = 0.851040526
